In [208]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
import os
import pandas as pd

In [209]:
def sentence_segmentation (string):
    segmented_string = nltk.sent_tokenize(string)
    return segmented_string

In [210]:
def string_without_punct(string):
    regex_punctuation = r'[^\w\s]'
    string_without_punct = re.sub(regex_punctuation,'',string)
    return string_without_punct

In [211]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.VERB)


In [212]:
def lemmatizer(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    
    for w in words:
        lemmas.append(lemmatizer.lemmatize(w, get_wordnet_pos(w)))
    return lemmas


In [213]:
def tokenizer(string):
    #https://www.w3schools.com/python/python_regex.asp#findall
    regex = r"\b[-a-zA-ZÀ-ÖØ-öø-ÿ0-9]+\b"
    return re.findall(regex, string)

In [214]:
def normalizer(word):
    return [ w.lower() for w in word]

In [215]:
class PorterStemmerCustom:
    #https://tartarus.org/martin/PorterStemmer/python.txt
    def isCons(self, letter):
        if letter == 'a' or letter == 'e' or letter == 'i' or letter == 'o' or letter == 'u':
            return False
        else:
            return True

    def isConsonant(self, word, i):
        letter = word[i]
        if self.isCons(letter):
            if letter == 'y' and self.isCons(word[i-1]):
                return False
            else:
                return True
        else:
            return False

    def isVowel(self, word, i):
        return not(self.isConsonant(word, i))

    # *S
    def endsWith(self, stem, letter):
        if stem.endswith(letter):
            return True
        else:
            return False

    # *v*
    def containsVowel(self, stem):
        for i in stem:
            if not self.isCons(i):
                return True
        return False

    # *d
    def doubleCons(self, stem):
        if len(stem) >= 2:
            if self.isConsonant(stem, -1) and self.isConsonant(stem, -2):
                return True
            else:
                return False
        else:
            return False

    def getForm(self, word):
        form = []
        formStr = ''
        for i in range(len(word)):
            if self.isConsonant(word, i):
                if i != 0:
                    prev = form[-1]
                    if prev != 'C':
                        form.append('C')
                else:
                    form.append('C')
            else:
                if i != 0:
                    prev = form[-1]
                    if prev != 'V':
                        form.append('V')
                else:
                    form.append('V')
        for j in form:
            formStr += j
        return formStr

    def getM(self, word):
        form = self.getForm(word)
        m = form.count('VC')
        return m

    # *o
    def cvc(self, word):
        if len(word) >= 3:
            f = -3
            s = -2
            t = -1
            third = word[t]
            if self.isConsonant(word, f) and self.isVowel(word, s) and self.isConsonant(word, t):
                if third != 'w' and third != 'x' and third != 'y':
                    return True
                else:
                    return False
            else:
                return False
        else:
            return False

    def replace(self, orig, rem, rep):
        result = orig.rfind(rem)
        base = orig[:result]
        replaced = base + rep
        return replaced

    def replaceM0(self, orig, rem, rep):
        result = orig.rfind(rem)
        base = orig[:result]
        if self.getM(base) > 0:
            replaced = base + rep
            return replaced
        else:
            return orig

    def replaceM1(self, orig, rem, rep):
        result = orig.rfind(rem)
        base = orig[:result]
        if self.getM(base) > 1:
            replaced = base + rep
            return replaced
        else:
            return orig

    def step1a(self, word):
        if word.endswith('sses'):
            word = self.replace(word, 'sses', 'ss')
        elif word.endswith('ies'):
            word = self.replace(word, 'ies', 'i')
        elif word.endswith('ss'):
            word = self.replace(word, 'ss', 'ss')
        elif word.endswith('s'):
            word = self.replace(word, 's', '')
        else:
            pass
        return word

    def step1b(self, word):
        flag = False
        if word.endswith('eed'):
            result = word.rfind('eed')
            base = word[:result]
            if self.getM(base) > 0:
                word = base
                word += 'ee'
        elif word.endswith('ed'):
            result = word.rfind('ed')
            base = word[:result]
            if self.containsVowel(base):
                word = base
                flag = True
        elif word.endswith('ing'):
            result = word.rfind('ing')
            base = word[:result]
            if self.containsVowel(base):
                word = base
                flag = True
        if flag:
            if word.endswith('at') or word.endswith('bl') or word.endswith('iz'):
                word += 'e'
            elif self.doubleCons(word) and not self.endsWith(word, 'l') and not self.endsWith(word, 's') and not self.endsWith(word, 'z'):
                word = word[:-1]
            elif self.getM(word) == 1 and self.cvc(word):
                word += 'e'
            else:
                pass
        else:
            pass
        return word

    def step1c(self, word):
        if word.endswith('y'):
            result = word.rfind('y')
            base = word[:result]
            if self.containsVowel(base):
                word = base
                word += 'i'
        return word

    def step2(self, word):
        if word.endswith('ational'):
            word = self.replaceM0(word, 'ational', 'ate')
        elif word.endswith('tional'):
            word = self.replaceM0(word, 'tional', 'tion')
        elif word.endswith('enci'):
            word = self.replaceM0(word, 'enci', 'ence')
        elif word.endswith('anci'):
            word = self.replaceM0(word, 'anci', 'ance')
        elif word.endswith('izer'):
            word = self.replaceM0(word, 'izer', 'ize')
        elif word.endswith('abli'):
            word = self.replaceM0(word, 'abli', 'able')
        elif word.endswith('alli'):
            word = self.replaceM0(word, 'alli', 'al')
        elif word.endswith('entli'):
            word = self.replaceM0(word, 'entli', 'ent')
        elif word.endswith('eli'):
            word = self.replaceM0(word, 'eli', 'e')
        elif word.endswith('ousli'):
            word = self.replaceM0(word, 'ousli', 'ous')
        elif word.endswith('ization'):
            word = self.replaceM0(word, 'ization', 'ize')
        elif word.endswith('ation'):
            word = self.replaceM0(word, 'ation', 'ate')
        elif word.endswith('ator'):
            word = self.replaceM0(word, 'ator', 'ate')
        elif word.endswith('alism'):
            word = self.replaceM0(word, 'alism', 'al')
        elif word.endswith('iveness'):
            word = self.replaceM0(word, 'iveness', 'ive')
        elif word.endswith('fulness'):
            word = self.replaceM0(word, 'fulness', 'ful')
        elif word.endswith('ousness'):
            word = self.replaceM0(word, 'ousness', 'ous')
        elif word.endswith('aliti'):
            word = self.replaceM0(word, 'aliti', 'al')
        elif word.endswith('iviti'):
            word = self.replaceM0(word, 'iviti', 'ive')
        elif word.endswith('biliti'):
            word = self.replaceM0(word, 'biliti', 'ble')
        return word

    def step3(self, word):
        if word.endswith('icate'):
            word = self.replaceM0(word, 'icate', 'ic')
        elif word.endswith('ative'):
            word = self.replaceM0(word, 'ative', '')
        elif word.endswith('alize'):
            word = self.replaceM0(word, 'alize', 'al')
        elif word.endswith('iciti'):
            word = self.replaceM0(word, 'iciti', 'ic')
        elif word.endswith('ful'):
            word = self.replaceM0(word, 'ful', '')
        elif word.endswith('ness'):
            word = self.replaceM0(word, 'ness', '')
        return word

    def step4(self, word):
        if word.endswith('al'):
            word = self.replaceM1(word, 'al', '')
        elif word.endswith('ance'):
            word = self.replaceM1(word, 'ance', '')
        elif word.endswith('ence'):
            word = self.replaceM1(word, 'ence', '')
        elif word.endswith('er'):
            word = self.replaceM1(word, 'er', '')
        elif word.endswith('ic'):
            word = self.replaceM1(word, 'ic', '')
        elif word.endswith('able'):
            word = self.replaceM1(word, 'able', '')
        elif word.endswith('ible'):
            word = self.replaceM1(word, 'ible', '')
        elif word.endswith('ant'):
            word = self.replaceM1(word, 'ant', '')
        elif word.endswith('ement'):
            word = self.replaceM1(word, 'ement', '')
        elif word.endswith('ment'):
            word = self.replaceM1(word, 'ment', '')
        elif word.endswith('ent'):
            word = self.replaceM1(word, 'ent', '')
        elif word.endswith('ou'):
            word = self.replaceM1(word, 'ou', '')
        elif word.endswith('ism'):
            word = self.replaceM1(word, 'ism', '')
        elif word.endswith('ate'):
            word = self.replaceM1(word, 'ate', '')
        elif word.endswith('iti'):
            word = self.replaceM1(word, 'iti', '')
        elif word.endswith('ous'):
            word = self.replaceM1(word, 'ous', '')
        elif word.endswith('ive'):
            word = self.replaceM1(word, 'ive', '')
        elif word.endswith('ize'):
            word = self.replaceM1(word, 'ize', '')
        elif word.endswith('ion'):
            result = word.rfind('ion')
            base = word[:result]
            if self.getM(base) > 1 and (self.endsWith(base, 's') or self.endsWith(base, 't')):
                word = base
            word = self.replaceM1(word, '', '')
        return word

    def step5a(self, word):
        if word.endswith('e'):
            base = word[:-1]
            if self.getM(base) > 1:
                word = base
            elif self.getM(base) == 1 and not self.cvc(base):
                word = base
        return word

    def step5b(self, word):
        if self.getM(word) > 1 and self.doubleCons(word) and self.endsWith(word, 'l'):
            word = word[:-1]
        return word

    def stem(self, word):
        word = self.step1a(word)
        word = self.step1b(word)
        word = self.step1c(word)
        word = self.step2(word)
        word = self.step3(word)
        word = self.step4(word)
        word = self.step5a(word)
        word = self.step5b(word)
        return word

In [216]:
def stemArray(words):
    psc = PorterStemmerCustom()
    result = []
    for w in words:
        result.append(psc.stem(w))
    return result    

In [217]:
class StopWordsHandler:
    #https://stackoverflow.com/questions/6022764/python-removing-list-element-while-iterating-over-list/6024599
    #https://gist.github.com/sebleier/554280
    def isStopWord(self,word):
        stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
        if word in stop_words:
            return True
        else:
            return False
    def removeStopWords(self,words):
        for i in list(words):
            if self.isStopWord(i) or i == '':
                words.remove(i)
            else:
                pass
        return words

In [218]:
def readCorpusData(path):
    files = []
    read_tuples = []
    #Lendo arquivos da pasta data/
    for r, d, f in os.walk(path):
        for file in f:
            #Caso seja .xls, eh o excel que possui os as respostas para o treinamento
            if '.xls' in file:
                xls = pd.ExcelFile(f'{path}{file}')
                #Transformar em DataFrame
                df = xls.parse('File list')
            elif '.txt' in file:
                with open(f'{path}{file}', 'r', encoding="utf8", errors='ignore') as j:
                    #Montar um lista de tuplas para mapear arquivo e conteudos
                    read_tuples.append((file, j.read()))

    #Transformar lista de tuplas em DF
    df_tuples = pd.DataFrame(read_tuples, columns=['File', 'Content'])
    #Realizar o Join entre os arquivos para mapear conteudo e respostas de treinamento
    return pd.merge(df_tuples, df, how='left', left_on = 'File', right_on = 'File')

In [219]:
path = 'data/'
corpusDF = readCorpusData(path)
corpusDF = corpusDF[['File', 'Category','Content']]
corpusDF.head()

,File,Category,Content
0,g0pA_taska.txt,non,Inheritance is a basic concept of Object-Orien...
1,g0pA_taskb.txt,cut,PageRank is a link analysis algorithm used by ...
2,g0pA_taskc.txt,light,"The vector space model (also called, term vect..."
3,g0pA_taskd.txt,heavy,Bayes’ theorem was names after Rev Thomas Baye...
4,g0pA_taske.txt,non,Dynamic Programming is an algorithm design tec...


In [220]:
taska = corpusDF[corpusDF.File.str.contains('taska.txt')].fillna(0)
taskb = corpusDF[corpusDF.File.str.contains('taskb.txt')].fillna(0)
taskc = corpusDF[corpusDF.File.str.contains('taskc.txt')].fillna(0)
taskd = corpusDF[corpusDF.File.str.contains('taskd.txt')].fillna(0)
taske = corpusDF[corpusDF.File.str.contains('taske.txt')].fillna(0)

In [221]:
taske['Tokens'] = taske['Content'].map(tokenizer).map(normalizer)

In [222]:
def wordCount(words):
    count= nltk.defaultdict(int)
    for word in words:
        count[word] += 1
    return count

In [223]:
def cos_dist(v1, v2):
    import numpy as np
    product = np.dot(v1,v2)
    
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    
    return product/(norm_v2*norm_v1)    

In [224]:
def getSim(t1, t2):
    import numpy as np
    vocabulary = []
    
    for key in t1:
        vocabulary.append(key)
    for key in t2:
        vocabulary.append(key)
    
    vocabulary_size = len(vocabulary)
    
    d1 = np.zeros(vocabulary_size, dtype=np.int)
    d2 = np.zeros(vocabulary_size, dtype=np.int)
    
    i = 0
    for (k) in vocabulary:
        d1[i] = t1.get(k, 0)
        d2[i] = t2.get(k, 0)
        i += 1

    return cos_dist(d1, d2)

In [225]:
taske['WCount'] = taske['Tokens'].map(wordCount)
taske[['File','Category']]

,File,Category
4,g0pA_taske.txt,non
9,g0pB_taske.txt,heavy
14,g0pC_taske.txt,light
19,g0pD_taske.txt,non
24,g0pE_taske.txt,cut
29,g1pA_taske.txt,non
34,g1pB_taske.txt,cut
39,g1pD_taske.txt,heavy
44,g2pA_taske.txt,non
49,g2pB_taske.txt,cut


In [226]:
def createF1(task):
    task['Tokens'] = task['Content'].map(tokenizer).map(normalizer)
    task['WCount'] = task['Tokens'].map(wordCount)
    task['f1'] = task.WCount.map(lambda t: getSim(task.WCount.iloc[-1], t))

In [227]:
def createF2(task):
    ps = PorterStemmerCustom()
    task['Tokens'] = task['Content'].map(tokenizer).map(normalizer)
    swh = StopWordsHandler()
    task['StopWordsOut'] = task['Tokens'].map(swh.removeStopWords)
    task['Stem'] = task['StopWordsOut'].map(stemArray)
    task['WCount'] = task['Stem'].map(wordCount)
    task['f2'] = task.WCount.map(lambda t: getSim(task.WCount.iloc[-1], t))

In [228]:
def createF3(task):
    ps = PorterStemmerCustom()
    task['Tokens'] = task['Content'].map(tokenizer).map(normalizer)
    swh = StopWordsHandler()
    task['StopWordsOut'] = task['Tokens'].map(swh.removeStopWords)
    task['Lem'] = task['StopWordsOut'].map(lemmatizer)
    task['WCount'] = task['Lem'].map(wordCount)
    task['f3'] = task.WCount.map(lambda t: getSim(task.WCount.iloc[-1], t))

In [229]:
def createF4(task):
    task['Tokens'] = task['Content'].map(tokenizer).map(normalizer)
    swh = StopWordsHandler()
    task['StopWordsOut'] = task['Tokens'].map(swh.removeStopWords)
    task['WCount'] = task['StopWordsOut'].map(wordCount)
    task['f4'] = task.WCount.map(lambda t: getSim(task.WCount.iloc[-1], t))

In [230]:
path = 'data/'
corpusDF = readCorpusData(path)
corpusDF = corpusDF[['File', 'Category','Content']]

In [231]:
taska = corpusDF[corpusDF.File.str.contains('taska.txt')].fillna(0)
taskb = corpusDF[corpusDF.File.str.contains('taskb.txt')].fillna(0)
taskc = corpusDF[corpusDF.File.str.contains('taskc.txt')].fillna(0)
taskd = corpusDF[corpusDF.File.str.contains('taskd.txt')].fillna(0)
taske = corpusDF[corpusDF.File.str.contains('taske.txt')].fillna(0)

In [232]:
createF1(taska)
createF1(taskb)
createF1(taskc)
createF1(taskd)
createF1(taske)

In [233]:
createF2(taska)
createF2(taskb)
createF2(taskc)
createF2(taskd)
createF2(taske)

In [234]:
createF3(taska)
createF3(taskb)
createF3(taskc)
createF3(taskd)
createF3(taske)

In [235]:
createF4(taska)
createF4(taskb)
createF4(taskc)
createF4(taskd)
createF4(taske)

In [239]:
taskd

,File,Category,Content,Tokens,WCount,f1,StopWordsOut,Stem,f2,Lem,f3,f4
3,g0pA_taskd.txt,heavy,Bayes’ theorem was names after Rev Thomas Baye...,"[bayes, theorem, names, rev, thomas, bayes, me...","{'bayes': 6, 'theorem': 5, 'names': 1, 'rev': ...",0.785225,"[bayes, theorem, names, rev, thomas, bayes, me...","[bay, theorem, name, rev, thoma, bay, method, ...",0.702416,"[bayes, theorem, name, rev, thomas, bayes, met...",0.690568,0.615714
8,g0pB_taskd.txt,light,Bayes' theorem relates the conditional and mar...,"[bayes, theorem, relates, conditional, margina...","{'bayes': 5, 'theorem': 6, 'relates': 1, 'cond...",0.924170,"[bayes, theorem, relates, conditional, margina...","[bay, theorem, relat, condit, margin, probab, ...",0.916154,"[bayes, theorem, relates, conditional, margina...",0.918430,0.920544
13,g0pC_taskd.txt,cut,In probability theory; Bayes theorem (often ca...,"[probability, theory, bayes, theorem, often, c...","{'probability': 5, 'theory': 1, 'bayes': 5, 't...",0.862990,"[probability, theory, bayes, theorem, often, c...","[probab, theori, bay, theorem, often, call, ba...",0.859161,"[probability, theory, bayes, theorem, often, c...",0.858417,0.802319
18,g0pD_taskd.txt,non,Baye’s theorm in connection with conditional p...,"[baye, theorm, connection, conditional, probab...","{'baye': 1, 'theorm': 1, 'connection': 1, 'con...",0.535447,"[baye, theorm, connection, conditional, probab...","[bay, theorm, connect, condit, probab, fundame...",0.337205,"[baye, theorm, connection, conditional, probab...",0.273117,0.173395
23,g0pE_taskd.txt,non,Bayes Theorem is an important theorem relating...,"[bayes, theorem, important, theorem, relating,...","{'bayes': 2, 'theorem': 3, 'important': 2, 're...",0.674645,"[bayes, theorem, important, theorem, relating,...","[bay, theorem, import, theorem, relat, condit,...",0.590174,"[bayes, theorem, important, theorem, relate, c...",0.586385,0.579106
28,g1pA_taskd.txt,cut,Bayes' theorem relates the conditional and mar...,"[bayes, theorem, relates, conditional, margina...","{'bayes': 4, 'theorem': 2, 'relates': 1, 'cond...",0.794802,"[bayes, theorem, relates, conditional, margina...","[bay, theorem, relat, condit, margin, probab, ...",0.757865,"[bayes, theorem, relates, conditional, margina...",0.760660,0.714148
33,g1pB_taskd.txt,light,Bayes theorem relates the conditional and marg...,"[bayes, theorem, relates, conditional, margina...","{'bayes': 3, 'theorem': 4, 'relates': 1, 'cond...",0.880557,"[bayes, theorem, relates, conditional, margina...","[bay, theorem, relat, condit, margin, probab, ...",0.812568,"[bayes, theorem, relates, conditional, margina...",0.816697,0.807665
38,g1pD_taskd.txt,non,Bayes Theorem is a mathematical formula used t...,"[bayes, theorem, mathematical, formula, used, ...","{'bayes': 3, 'theorem': 3, 'mathematical': 1, ...",0.805268,"[bayes, theorem, mathematical, formula, used, ...","[bay, theorem, mathematic, formula, us, calcul...",0.718071,"[bayes, theorem, mathematical, formula, use, c...",0.712328,0.674329
43,g2pA_taskd.txt,cut,"In probability theory, Bayes' theorem (often ...","[probability, theory, bayes, theorem, often, c...","{'probability': 10, 'theory': 1, 'bayes': 6, '...",0.828696,"[probability, theory, bayes, theorem, often, c...","[probab, theori, bay, theorem, often, call, ba...",0.714393,"[probability, theory, bayes, theorem, often, c...",0.716282,0.713141
48,g2pB_taskd.txt,light,"\nIn probability theory, Bayes' theorem also c...","[probability, theory, bayes, theorem, also, ca...","{'probability': 9, 'theory': 1, 'bayes': 7, 't...",0.971712,"[probability, theory, bayes, theorem, also, ca...","[probab, theori, bay, theorem, also, call, bay...",0.958509,"[probability, theory, bayes, theorem, also, ca...",0.958304,0.948742


In [237]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

#X nossos dados
X = 

#Y variável resposta, no caso nível de plágio
Y = 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 

gauss = GaussianNB()

gauss.fit(X_Train, y_train)

y_pred = gauss.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred)

print(f'Acurácia do classificador Naive Bayes: {(accuracy*100)}%')

SyntaxError: invalid syntax (<ipython-input-237-3059e618ba7b>, line 6)